### TODO:
1. Manually change "-ku" to actual city
2. Osaka vs Ōsaka
3. Change all prefecture 13 city to Tokyo
4. Some city are represented by List
5. Kyoto vs Kyōto
6. `"city": ".*Ō.*"`
7. Ōdate, 

In [1]:
import json
# from fuzzywuzzy import fuzz
import requests
from requests.exceptions import HTTPError
import time

In [2]:
stationsF = open("translated_stations_test.json")
stations = json.load(stationsF)

In [3]:
def normalize_long_vowels(text: str):
    return text.replace("Ō", "O").replace("ō", "o").replace("Ū", "U").replace("ū", "u")

In [ ]:
for s in stations:
    if isinstance(s['city'], list):
        if len(s['city']) == 1:
            # If there's only one item in the list, assign it as the city name
            s['city'] = normalize_long_vowels(s['city'][0])
        elif len(s['city']) > 1:
            # If there are multiple items in the list, set 'city' to None
            s['city'] = None
    if isinstance(s['city'], str):
        s['city'] = normalize_long_vowels(s['city'])
    # Extreme special case
    if s['city'] == 'Koda':
        s['city'] = 'Koda'
    if s['city'] == 'Nan\'yo':
        s['city'] = 'Nanyo'


In [23]:
with open("translated_stations_normalized.json", "w", encoding="utf-8") as of:
    json.dump(stations, of, ensure_ascii=False, indent=4)

### Add prefectures info to shinkansen

TODO: There are stations with same name in different prefectures. This leads to error in the "translated_stations_normalized.json" data. Need to redo the addition. (Manually modified JSON)

In [24]:
stations_to_add = []
s1 = ["東京", "品川", "新横浜", "小田原", "熱海", "三島", "新富士", "静岡", "掛川", "浜松", "豊橋", "三河安城", "名古屋", "岐阜羽島", "米原", "京都", "新大阪"]
stations_to_add.append(s1)
s2 = ["新大阪", "新神戸", "西明石", "姫路", "相生", "岡山", "新倉敷", "福山", "新尾道", "三原", "東広島", "広島", "新岩国", "徳山", "新山口", "厚狭", "新下関", "小倉", "博多"]
stations_to_add.append(s2)
s3 = ["東京", "上野", "大宮", "小山", "宇都宮", "那須塩原", "新白河", "郡山", "福島", "白石蔵王", "仙台", "古川", "くりこま高原", "一ノ関", "水沢江刺", "北上", "新花巻", "盛岡", "いわて沼宮内", "二戸", "八戸", "七戸十和田", "新青森"]
stations_to_add.append(s3)
s4 = ["東京", "上野", "大宮", "熊谷", "本庄早稲田", "高崎", "上毛高原", "越後湯沢", "浦佐", "長岡", "燕三条", "新潟"]
stations_to_add.append(s4)
s5 = []
# s5 = ["越後湯沢", "ガーラ湯沢"]
stations_to_add.append(s5)
s6 = ["福島", "米沢", "高畠", "赤湯", "かみのやま温泉", "山形", "天童", "さくらんぼ東根", "村山", "大石田", "新庄"]
stations_to_add.append(s6)
s7 = ["東京", "上野", "大宮", "小山", "宇都宮", "那須塩原", "新白河", "郡山", "福島", "白石蔵王", "仙台", "古川", "くりこま高原", "一ノ関", "水沢江刺", "北上", "新花巻", "盛岡", "雫石", "田沢湖", "角館", "大曲", "秋田"]
stations_to_add.append(s7)
s8 = ["東京", "上野", "大宮", "熊谷", "本庄早稲田", "高崎", "安中榛名", "軽井沢", "佐久平", "上田", "長野", "飯山", "上越妙高", "糸魚川", "黒部宇奈月温泉", "富山", "新高岡", "金沢", "小松", "加賀温泉", "芦原温泉", "福井", "越前たけふ", "敦賀", "東小浜", "京都", "松井山手", "新大阪"]
stations_to_add.append(s8)
s9 = ["博多", "新鳥栖", "久留米", "筑後船小屋", "新大牟田", "新玉名", "熊本", "新八代", "新水俣", "出水", "川内", "鹿児島中央"]
stations_to_add.append(s9)
s10 = ["新青森", "奥津軽いまべつ", "木古内", "新函館北斗"]
stations_to_add.append(s10)

In [53]:
stationsF = open("translated_stations_normalized.json")
stations = json.load(stationsF)

In [55]:
def add_shinkansen_prefectures(stations: list[str], line: dict, stations_data_json: list[dict]):
    line_name = line["name_kanji"]
    line_id = line["ekidata_id"]
    prefectures = set()
    if stations is None:
        return

    for i, st in enumerate(stations):
        found = False
        for station_data in stations_data_json:
            # Check if station name matches any entry in station_data
            if station_data.get("name_kanji") == st or st in station_data.get("alternative_names", []):
                if found:
                    print(f"Station {st} was processed twice for line {line_name}, id {line_id}")
                    # break
                    continue
                found = True
                if line_id not in station_data["ekidata_line_ids"]:
                    print(f"Station {st} in prefecture {station_data['prefecture']} is passed")
                    continue
                # Get details for the existing station entry
                cur_prefecture = station_data["prefecture"]
                # print(f"For line {line_name}, station {st} is found in prefecture {cur_prefecture}")
                prefectures.add(cur_prefecture)
                line["prefectures"] = list(prefectures)
                
                # Break after finding and processing the correct station
                # break

        if not found:
            print(f"Station {st} is not found for line {line_name}")

In [56]:
linesF = open("processed_lines.json")
lines = json.load(linesF)

In [57]:
for i in range(10):
    add_shinkansen_prefectures(stations_to_add[i], lines[i], stations)

Station 新富士 in prefecture 01 is passed
Station 岐阜羽島 is not found for line 東海道新幹線
Station 新尾道 is not found for line 山陽新幹線
Station 東広島 is not found for line 山陽新幹線
Station 新岩国 is not found for line 山陽新幹線
Station 郡山 was processed twice for line 東北新幹線, id 1004
Station 福島 was processed twice for line 東北新幹線, id 1004
Station 白石蔵王 is not found for line 東北新幹線
Station くりこま高原 is not found for line 東北新幹線
Station 水沢江刺 is not found for line 東北新幹線
Station 七戸十和田 is not found for line 東北新幹線
Station 本庄早稲田 is not found for line 上越新幹線
Station 上毛高原 is not found for line 上越新幹線
Station 福島 was processed twice for line 山形新幹線, id 1007
Station 郡山 was processed twice for line 秋田新幹線, id 1008
Station 福島 was processed twice for line 秋田新幹線, id 1008
Station 白石蔵王 is not found for line 秋田新幹線
Station くりこま高原 is not found for line 秋田新幹線
Station 水沢江刺 is not found for line 秋田新幹線
Station 本庄早稲田 is not found for line 北陸新幹線
Station 安中榛名 is not found for line 北陸新幹線
Station 黒部宇奈月温泉 is not found for line 北陸新幹線
Station 越前たけふ is not f

In [58]:
for i in range(10):
    print(json.dumps(lines[i], ensure_ascii=False, indent=4))

{
    "code": "",
    "ekidata_id": "1002",
    "name_kanji": "東海道新幹線",
    "name_kana": "とうかいどうしんかんせん",
    "name_romaji": "Tokaido Shinkansen",
    "alternative_names": [
        "トウカイドウシンカンセン"
    ],
    "prefectures": [
        "25",
        "22",
        "14",
        "27",
        "13",
        "26",
        "23"
    ],
    "logo": ""
}
{
    "code": "",
    "ekidata_id": "1003",
    "name_kanji": "山陽新幹線",
    "name_kana": "さんようしんかんせん",
    "name_romaji": "San'yo Shinkansen",
    "alternative_names": [
        "サンヨウシンカンセン"
    ],
    "prefectures": [
        "40",
        "28",
        "34",
        "35",
        "27",
        "33"
    ],
    "logo": ""
}
{
    "code": "",
    "ekidata_id": "1004",
    "name_kanji": "東北新幹線",
    "name_kana": "とうほくしんかんせん",
    "name_romaji": "Tohoku Shinkansen",
    "alternative_names": [
        "トウホクシンカンセン"
    ],
    "prefectures": [
        "02",
        "07",
        "13",
        "04",
        "03",
        "09",
        "11"
    ],
    "log

In [59]:
with open("processed_lines.json", "w", encoding="utf-8") as of:
    json.dump(lines, of, ensure_ascii=False, indent=4)